In [11]:
import pandas as pd
import datetime as dt

#grab flight plan
flight_plan = pd.read_csv(r'/Users/apple/Desktop/Barrior/csv/iss_flight_plan_20220101-20251231.csv')


#query to only use dates where the event is dock and convert it to a list
dock_days_list = flight_plan[flight_plan['event'] == 'Dock']['datedim'].to_list()

#convert list of date strings into a list of datetime objects
dates_list = [dt.datetime.strptime(date, '%m/%d/%Y').date() for date in dock_days_list]

#print("dates list\n",dates_list)
#print the days difference between each date
num_days = pd.Series(dates_list).diff().dt.days.iloc[1:].astype(int).tolist()
print("number of daays til next dock\n",num_days)

flight_plan['datedim'] = pd.to_datetime(flight_plan['datedim'])

dock_days_list_next2years = flight_plan[
    (flight_plan['datedim'].dt.year >= 2024) & 
    (flight_plan['datedim'].dt.year <= 2025) & 
    (flight_plan['event'] == 'Dock')
]['datedim'].to_list()
#print("number of days til next 2 years\n",dock_days_list_next2years)
#print(len (dock_days_list))
#print("number of days til next 2 years\n",len(dock_days_list_next2years))



number of daays til next dock
 [4, 25, 22, 18, 24, 13, 43, 67, 15, 22, 12, 18, 76, 15, 5, 13, 67, 2, 13, 59, 21, 2, 19, 48, 29, 41, 10, 10, 17, 2, 15, 8, 30, 22, 7, 22, 29, 35, 16, 6, 14, 9, 44, 33, 4, 22, 1, 34, 3, 19, 17, 62, 4, 27, 48, 2, 29, 7, 25, 28, 25, 25]


In [12]:
#grab crew members per dock
crewF = pd.read_csv(r'/Users/apple/Desktop/Barrior/csv/iss_flight_plan_crew_20220101-20251321.csv')

crew_per_dock = []

for i, date in enumerate(dock_days_list):
    crewNum = crewF[crewF['datedim'] == date]['crew_count'].tolist()
    Amount = sum(crewNum)
    crew_per_dock.append(Amount)

#number of crew members on board during each dock
print("number of crew per docking\n",crew_per_dock)
#print(len(crew_per_dock))

number of crew per docking
 [7, 7, 10, 11, 11, 7, 7, 7, 10, 11, 7, 7, 7, 7, 7, 11, 7, 11, 11, 7, 7, 7, 11, 10, 7, 7, 11, 11, 7, 7, 11, 7, 10, 9, 7, 11, 7, 7, 11, 7, 7, 10, 10, 11, 7, 7, 7, 7, 7, 11, 7, 10, 11, 11, 7, 7, 11, 7, 10, 7, 7, 11, 7]


In [13]:
#grab rates
rater = pd.read_csv(r'/Users/apple/Desktop/Barrior/csv/rates_definition.csv')


#consumable must match a consumable on the rates csv. case sensitive.
consumable = 'Nitrogen'
#consumable = 'Air'
#consumable = 'KTO'
#crew_used = 2.7

#sum of generated
sum_generated = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'generation')]['rate'].to_list())

#grabs the usage rate of consumable and sums it up
sum_usage = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'usage')]['rate'].to_list())

#if units contains 'crew'
crew_check = rater[rater['affected_consumable'] == consumable]['units'].tolist()
combine =  '\t'.join(crew_check)

#print(crew_check)
if 'Crew' in combine:
    per_crew = True
else:
    per_crew = False

#print(per_crew)

#prints
print("Consumable used = ", sum_usage)
print("Consumable generated = ", sum_generated)
#net_consumable = sum_generated - sum_usage
#print("net consumable per day= ", net_consumable)



Consumable used =  0.48492900000000005
Consumable generated =  0


In [14]:
import math
i#mport numbers

#create a list of base rates 
listofrates = [sum_usage] * len(dock_days_list)
#print(len(dock_days_list))
consumable_to_send = []
consumable_water_to_send = []
consumable_water_to_send_new = []
consumable_to_send_water = []
current_sum = 0
current_sum1 = 0
current_sum2 = 0
current_sum3 = 0
res_list = []

if(per_crew): #if percrew true change the list of rates
    for i in range(0, len(listofrates)):
        res_list.append(listofrates[i] * crew_per_dock[i])
else:
    res_list = listofrates
#print(res_list)
#match length and subtract the amount generated to get proper list of rates   
res_list.pop()
res_list = [x - sum_generated for x in res_list]

intermediate = []
for i in range(0, len(res_list)):
    intermediate.append(res_list[i] * num_days[i])
print ("The amount need to send = ",intermediate)

multiplied_crew_numbers = []

for i in range(len(crew_per_dock)):
    multiplied_crew_numbers.append(crew_per_dock[i] * 2.7)
#print("amount of water use by crew = ",multiplied_crew_numbers)

#rounding
for i, value in enumerate(intermediate):
    current_sum+=value
    consumable_to_send.append(math.trunc(current_sum))
    current_sum = current_sum - math.trunc(current_sum)
#print("amount of water need to send = ",consumable_to_send)
#print (len(consumable_to_send))

#rounding amount of water for crew
for i, value in enumerate(multiplied_crew_numbers):
    current_sum1+=value
    consumable_water_to_send.append(math.trunc(current_sum1))
    current_sum1 = current_sum1 - math.trunc(current_sum1)
#print("amount of water use by crew = ",consumable_water_to_send)

need_to_send_list = []

#for i in range(len(consumable_to_send)):
 #   x = consumable_to_send[i] + (sum_generated * num_days[i])
  #  x = x / sum_usage
   # x = x + (sum_usage - 5.34) * num_days[i]   # new step # overall use 11.5 * 4 = 46
    #need_to_send = x * crew_used
    #need_to_send_list.append(need_to_send)
#print("amount of water need to send = ",need_to_send_list)



#rounding amount of water for crew
#for i, value in enumerate(need_to_send_list):
#    current_sum2+=value
#    consumable_water_to_send_new.append(math.trunc(current_sum2))
#   current_sum2 = current_sum2 - math.trunc(current_sum2)
#print("amount of water use by crew = ",consumable_water_to_send_new)
#print (len(consumable_water_to_send_new))


for i in range(len(intermediate)):
    x = sum_usage * num_days[i]
    #print(x)
    #y = 23.55 * num_days[i]
    #print(y)
    #x = x + y
    #print(x)
    #y = sum_generated * num_days[i]
    #print(y)
    #need_to_send = x + y
    #print(need_to_send)
    need_to_send_list.append(x)

#print("amount of water need to send = ",need_to_send_list)
#print (len(need_to_send_list))
for i, value in enumerate(need_to_send_list):
    current_sum3+=value
    consumable_to_send_water.append(math.trunc(current_sum3))
    current_sum3 = current_sum3 - math.trunc(current_sum3)
#print("amount of water need to send = ",consumable_to_send_water)
#print (len(consumable_to_send))



greatest_number = max(need_to_send_list[26:62])
greatest_number = round(greatest_number,2)
print("The greatest number in the array is:", greatest_number, "lbs")

#index_of_greatest_number = need_to_send_list.index(greatest_number)
#print("The index of the greatest number is:", index_of_greatest_number)

sum_number = round(sum(need_to_send_list[26:62]),2)
print("the sum amount of the next two years",sum_number, "lbs")
#print(need_to_send_list[61])


sum_number2 = round(sum(need_to_send_list[0:26]),2)
print("the sum amount of the historical usage",sum_number2, "lbs")

the_difference = round(sum_number - sum_number2,2)
print("the difference between the next two years and the historical usage: ",the_difference, "lbs")

the_difference_percentage = round((the_difference/sum_number2) * 100,2)
#print(the_difference/sum_number2)
print("the difference percentage between the next two years and the historical usage: ",the_difference_percentage,"%")

# 2/17/2022
# sub by 7 x 16.84 = 117.88


# use 2.7 x 7 * 4 = 75.6
# overall use 11.5 * 4 = 46
# total use 75.6 + 46 = 121.6 
# generate 23.55 * 4 = 94.2
# net 121.6 - 94.2 = 27.4

# 4 * 7 * 16.84 = 471.52 //121.6 
# 23.55 * 4 = 94.2
# net total = 471.52 - 94.2 = 377.32

# 377 + 94.2 = 471.2 / 16.84 = 28.01 * 2.7 = 75.6 + 11.5 * 4 = 121.6

The amount need to send =  [1.9397160000000002, 12.123225000000001, 10.668438000000002, 8.728722000000001, 11.638296, 6.304077, 20.851947000000003, 32.49024300000001, 7.273935000000001, 10.668438000000002, 5.819148, 8.728722000000001, 36.854604, 7.273935000000001, 2.4246450000000004, 6.304077, 32.49024300000001, 0.9698580000000001, 6.304077, 28.610811, 10.183509, 0.9698580000000001, 9.213651, 23.276592, 14.062941000000002, 19.882089, 4.849290000000001, 4.849290000000001, 8.243793, 0.9698580000000001, 7.273935000000001, 3.8794320000000004, 14.547870000000001, 10.668438000000002, 3.3945030000000003, 10.668438000000002, 14.062941000000002, 16.972515, 7.758864000000001, 2.909574, 6.7890060000000005, 4.364361000000001, 21.336876000000004, 16.002657000000003, 1.9397160000000002, 10.668438000000002, 0.48492900000000005, 16.487586, 1.454787, 9.213651, 8.243793, 30.065598000000005, 1.9397160000000002, 13.093083000000002, 23.276592, 0.9698580000000001, 14.062941000000002, 3.3945030000000003, 12.

In [15]:
##RUN ALL or this wont work
dock_days_list.pop()
#print(len(dock_days_list))
readable = pd.DataFrame(
    {'Docking Date': dock_days_list,
     'Estimation of Sent Consumables': need_to_send_list
    })
#print(len(dock_days_list))
#print(len(need_to_send_list))
print(dock_days_list[51])
print("( 62 ) days remaining before the next dock is dispatched and a workforce of ( 11 ) crews.")
#display(readable)
#data = pd.read_csv(readable)
pd.set_option('display.max_rows', None)
display(readable)

3/15/2025
( 62 ) days remaining before the next dock is dispatched and a workforce of ( 11 ) crews.


,Docking Date,Estimation of Sent Consumables
0,2/17/2022,1.939716
1,2/21/2022,12.123225
2,3/18/2022,10.668438
3,4/9/2022,8.728722
4,4/27/2022,11.638296
5,5/21/2022,6.304077
6,6/3/2022,20.851947
7,7/16/2022,32.490243
8,9/21/2022,7.273935
9,10/6/2022,10.668438
